# Simulating Many Sources in `castor_etc`

Isaac Cheng - April 2022

This notebook shows how to process many point sources using the `castor_etc` Python
package. Note that we do not yet support generating multiple sources in the same aperture
(but you can always simulate your own and [use your own data](./custom_source.ipynb) with
the ETC!).


In [1]:
import astropy.units as u
import numpy as np
import pandas as pd
from IPython.display import display

from castor_etc.background import Background
from castor_etc.photometry import Photometry
from castor_etc.sources import PointSource
from castor_etc.telescope import Telescope

We will use the same `Telescope` and (sky) `Background` objects for all of our photometry
calculations.


In [2]:
MyTelescope = Telescope()
MyBackground = Background()
MyBackground.add_geocoronal_emission("avg")

Now we will generate many different point sources at different AB magnitudes, then find
the time required (in seconds) to achieve a signal-to-noise ratio of 5 and 10 in each
passband.


In [3]:
TARGET_SNRS = [5, 10]
AB_MAGS = np.arange(22, 29, 0.5)
REDDENING = 0.0

wavelengths = np.arange(90, 1200) * u.nm  # generate the spectrum over these wavelengths

# Make some dictionaries to store the results of our calculations
times_to_one_snr = dict.fromkeys(AB_MAGS)
times_to_reach_snrs = {snr: times_to_one_snr.copy() for snr in TARGET_SNRS}

for ab_mag in AB_MAGS:
    #
    # Initialize a point source and generate a flat spectrum in units of AB magnitude
    #
    MySource = PointSource()
    MySource.generate_uniform(wavelengths=wavelengths, value=ab_mag, unit="ABmag")
    #
    # Initialize a Photometry object and use an optimal aperture for a point source
    #
    MyPhot = Photometry(MyTelescope, MySource, MyBackground)
    MyPhot.use_optimal_aperture(quiet=True)  # don't print encircled energy fraction
    for snr in TARGET_SNRS:
        #
        # Find the time, in seconds, required to reach the specified signal-to-noise ratio
        #
        times_to_reach_snrs[snr][ab_mag] = MyPhot.calc_snr_or_t(
            snr=snr, reddening=REDDENING, quiet=True
        )

In [4]:
for snr in TARGET_SNRS:
    print(f"Times (s) required to reach S/N = {snr}")
    display(pd.DataFrame(times_to_reach_snrs[snr]).round(2).T)

Times (s) required to reach S/N = 5


,uv,u,g
22.0,4.44,4.14,3.42
22.5,7.04,6.57,5.46
23.0,11.17,10.44,8.76
23.5,17.74,16.61,14.15
24.0,28.19,26.50,23.09
24.5,44.86,42.44,38.30
25.0,71.58,68.37,65.07
25.5,114.63,111.17,114.42
26.0,184.68,183.34,210.75
26.5,300.30,308.87,410.82


Times (s) required to reach S/N = 10


,uv,u,g
22.0,14.04,13.09,10.88
22.5,22.28,20.80,17.43
23.0,35.36,33.10,28.11
23.5,56.18,52.78,45.76
24.0,89.38,84.45,75.59
24.5,142.53,135.88,127.60
25.0,228.08,220.48,222.03
25.5,367.00,362.38,402.38
26.0,595.55,607.11,766.60
26.5,979.01,1045.49,1543.64
